In [29]:

from statistics import mean
import numpy as np
import itertools
from collections import defaultdict
import numpy as np
import itertools
from statistics import mean
import os
import time
import json
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import traceback
import logging
import pandas as pd
import multiprocessing
import os
import time
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [30]:
class SIFt(object):
    def __init__(self, pickled_sift):
        
        if pickled_sift is None:
            print("ohno")
            return None

        """
        Initialize an SIFt class.
        Parameters:
            fn_pro_PDB - PDB file name of the protein
            fn_lig_PDB - PDB file name of the ligand
            fn_pro_MOL2 - MOL2 file name of the protein
            fn_lig_MOL2 - MOL2 file name of the ligand
            ID - ID of the complex
            addH - whether to add hydrogen atoms when reading in the structure file
            sant - whether to sanitize the molecule when reading in the structure file
            int_cutoff - distance threshold for identifying protein-ligand interacting atoms 
        """
        self.ID = pickled_sift.ID 
       # print('Constructing an SIFt object for %s.........\n' % self.ID)
        # read in pdb coordinates and topology
        self.lig = pickled_sift.lig
        self.pro = pickled_sift.pro

 
       
        # parse protein pdb file for identifying sidechain/mainchain atoms
#        parser = PDBParser()
#        self.structure = parser.get_structure(self.ID, fn_pro_PDB)
#        self.chid = self.pro[1].GetAtomWithIdx(0).GetPDBResidueInfo().GetChainId()
        # identify interacting area
        self.contact_bins = pickled_sift.contact_bins
        self.pd = pickled_sift.pd
        self.cont = pickled_sift.cont
        self.contacts = pickled_sift.contacts

        self.protein_env = pickled_sift.protein_env
        self.ligand_env = pickled_sift.ligand_env
        
    #IMCP
    def get_quasi_fragmental_desp_ext1(self, bins = None):

  
        """
        Compute extended quasi fragmental descriptors.
        Parameters:
            bins - distance bins for extracting qf descriptors
        """
        if bins is not None:
            self.contact_bins = bins
        else:
            self.contact_bins = [(0, 12)]

        # list atom types ----------------------------------------------------
        pro_pool = [6, 7, 8, 16]
        lig_pool = [6, 7, 8, 9, 15, 16, 17, 35, 53]
        # get list of atom-pair types ----------------------------------------
        quasi_type = list(itertools.product(pro_pool, lig_pool))        
        quasi_feat = []

        for cbin in self.contact_bins:
            occur = {}
            for tp in quasi_type:
                occur[tp] = [0, []]
            # check the contacts one by one ----------------------------------
            contacts = np.nonzero((self.pd >= cbin[0]) & (self.pd < cbin[1]))
            conts = [(int(i), int(j)) for (i, j) in zip(contacts[0], contacts[1])]
            distances = [self.pd[i, j] for (i, j) in conts]
            for ind in range(len(conts)):
                cont = conts[ind]
                cur_dist = distances[ind]
                atm1 = self.pro[1].GetAtomWithIdx(cont[0])
                atm2 = self.lig[1].GetAtomWithIdx(cont[1])                
                atm1_an = atm1.GetAtomicNum()
                atm2_an = atm2.GetAtomicNum()
                
                tmp = (atm1_an, atm2_an)
                if tmp in quasi_type:
                    occur[tmp][0] += 1
                    occur[tmp][1] += [cur_dist]
            
            for tp in quasi_type:
                if occur[tp][0] == 0:
                    quasi_feat += [0, 0]
                else:
                    quasi_feat += [occur[tp][0], mean(occur[tp][1])]
        return quasi_feat  

    #IMC

    def get_quasi_fragmental_desp_ext2(self, bins = None):
        """
        Compute extended quasi fragmental descriptors.
        Parameters:
            bins - distance bins for extracting qf descriptors
        """
        if bins is not None:
            self.contact_bins = bins
        else:
            self.contact_bins = [(0, 12)]

        # list atom types ----------------------------------------------------
        pro_pool = [6, 7, 8, 16]
        lig_pool = [6, 7, 8, 9, 15, 16, 17, 35, 53]
        # get list of atom-pair types ----------------------------------------
        quasi_type = list(itertools.product(pro_pool, lig_pool))        
        quasi_feat = []

        for cbin in self.contact_bins:
            occur = {}
            for tp in quasi_type:
                occur[tp] = [0, []]
            # check the contacts one by one ----------------------------------
            contacts = np.nonzero((self.pd >= cbin[0]) & (self.pd < cbin[1]))
            conts = [(int(i), int(j)) for (i, j) in zip(contacts[0], contacts[1])]
            distances = [self.pd[i, j] for (i, j) in conts]
            for ind in range(len(conts)):
                cont = conts[ind]
                cur_dist = distances[ind]
                atm1 = self.pro[1].GetAtomWithIdx(cont[0])
                atm2 = self.lig[1].GetAtomWithIdx(cont[1])                
                atm1_an = atm1.GetAtomicNum()
                atm2_an = atm2.GetAtomicNum()
                
                tmp = (atm1_an, atm2_an)
                if tmp in quasi_type:
                    occur[tmp][0] += 1
                    occur[tmp][1] += [cur_dist]
            
            for tp in quasi_type:
                if occur[tp][0] == 0:
                    quasi_feat += [0]
                else:
                    quasi_feat += [occur[tp][0]]
        return quasi_feat  


    # protien env weighted
    def get_quasi_fragmental_desp_ext3(self, bins = None):
            """
            Compute extended quasi fragmental descriptors.
            Parameters:
                bins - distance bins for extracting qf descriptors
            """
            if bins is not None:
                self.contact_bins = bins
            else:
                self.contact_bins = [(0, 12)]

            # list atom types ----------------------------------------------------
            pro_pool = [6, 7, 8, 16]
            lig_pool = [6, 7, 8, 9, 15, 16, 17, 35, 53]
            # get list of atom-pair types ----------------------------------------
            quasi_type = list(itertools.product(pro_pool, lig_pool))        
            quasi_feat = []
            debug = defaultdict(int)
            for cbin in self.contact_bins:
                occur = {}
                for tp in quasi_type:
                    occur[tp] = [0, [],[]]
                # check the contacts one by one ----------------------------------
                contacts = np.nonzero((self.pd >= cbin[0]) & (self.pd < cbin[1]))
                conts = [(int(i), int(j)) for (i, j) in zip(contacts[0], contacts[1])]
                distances = [self.pd[i, j] for (i, j) in conts]
                for ind in range(len(conts)):
                    cont = conts[ind]
                    cur_dist = distances[ind]
                    atm1 = self.pro[1].GetAtomWithIdx(cont[0])
                    atm2 = self.lig[1].GetAtomWithIdx(cont[1])                
                    atm1_an = atm1.GetAtomicNum()
                    atm2_an = atm2.GetAtomicNum()
                    
                    tmp = (atm1_an, atm2_an)
                    if tmp in quasi_type:
                        occur[tmp][0] += 1
                        env1 = self.protein_env.get(cont[0], 1)
                        env2 = self.ligand_env.get(cont[1], 1)
                        debug[(env1,env2)] += 1
                        occur[tmp][1] += [ ( 1/env1 ) * cur_dist]
                        occur[tmp][2] += [( 1/env1 ) ]
                
                for tp in quasi_type:
                    if occur[tp][0] == 0:
                        quasi_feat += [0, 0]
                    else:
                        quasi_feat += [occur[tp][0], sum(occur[tp][1])/ sum(occur[tp][2])]

            return quasi_feat  

    # IMCEP
    def get_quasi_fragmental_desp_ext4(self, bins = None):
            """
            Compute extended quasi fragmental descriptors.
            Parameters:
                bins - distance bins for extracting qf descriptors
            """
            if bins is not None:
                self.contact_bins = bins
            else:
                self.contact_bins = [(0, 12)]

            # list atom types ----------------------------------------------------
            pro_pool = [6, 7, 8, 16]
            lig_pool = [6, 7, 8, 9, 15, 16, 17, 35, 53]
            # get list of atom-pair types ----------------------------------------
            quasi_type = list(itertools.product(pro_pool, lig_pool))        
            quasi_feat = []
            debug = defaultdict(int)
            for cbin in self.contact_bins:
                occur = {}
                for tp in quasi_type:
                    occur[tp] = [0, [], [], []]
                # check the contacts one by one ----------------------------------
                contacts = np.nonzero((self.pd >= cbin[0]) & (self.pd < cbin[1]))
                conts = [(int(i), int(j)) for (i, j) in zip(contacts[0], contacts[1])]
                distances = [self.pd[i, j] for (i, j) in conts]
                for ind in range(len(conts)):
                    cont = conts[ind]
                    cur_dist = distances[ind]
                    atm1 = self.pro[1].GetAtomWithIdx(cont[0])
                    atm2 = self.lig[1].GetAtomWithIdx(cont[1])                
                    atm1_an = atm1.GetAtomicNum()
                    atm2_an = atm2.GetAtomicNum()
                    
                    tmp = (atm1_an, atm2_an)
                    if tmp in quasi_type:
                        occur[tmp][0] += 1
                        env1 = self.protein_env.get(cont[0], 1)
                        env2 = self.ligand_env.get(cont[1], 1)
                        debug[(env1,env2)] += 1
                        occur[tmp][1] += [cur_dist]
                        occur[tmp][2] += [ 1/env1 ]
                        occur[tmp][3] += [ env2 ]
                
                for tp in quasi_type:
                    if occur[tp][0] == 0:
                        quasi_feat += [0,0,0]
                    else:
                        quasi_feat += [occur[tp][0], mean(occur[tp][1]),mean(occur[tp][2])]

            return quasi_feat  

    #IMCPiEB
    def get_quasi_fragmental_desp_ext5(self, bins = None):
            """
            Compute extended quasi fragmental descriptors.
            Parameters:
                bins - distance bins for extracting qf descriptors
            """
            if bins is not None:
                self.contact_bins = bins
            else:
                self.contact_bins = [(0, 12)]

            # list atom types ----------------------------------------------------
            pro_pool = [6, 7, 8, 16]
            lig_pool = [6, 7, 8, 9, 15, 16, 17, 35, 53]
            # get list of atom-pair types ----------------------------------------
            quasi_type = list(itertools.product(pro_pool, lig_pool))        
            quasi_feat = []
            debug = defaultdict(int)
            for cbin in self.contact_bins:
                occur = {}
                for tp in quasi_type:
                    occur[tp] = [0, [], [], []]
                # check the contacts one by one ----------------------------------
                contacts = np.nonzero((self.pd >= cbin[0]) & (self.pd < cbin[1]))
                conts = [(int(i), int(j)) for (i, j) in zip(contacts[0], contacts[1])]
                distances = [self.pd[i, j] for (i, j) in conts]
                for ind in range(len(conts)):
                    cont = conts[ind]
                    cur_dist = distances[ind]
                    atm1 = self.pro[1].GetAtomWithIdx(cont[0])
                    atm2 = self.lig[1].GetAtomWithIdx(cont[1])                
                    atm1_an = atm1.GetAtomicNum()
                    atm2_an = atm2.GetAtomicNum()
                    
                    tmp = (atm1_an, atm2_an)
                    if tmp in quasi_type:
                        occur[tmp][0] += 1
                        env1 = self.protein_env.get(cont[0], 1)
                        env2 = self.ligand_env.get(cont[1], 1)
                        debug[(env1,env2)] += 1
                        occur[tmp][1] += [cur_dist]
                        occur[tmp][2] += [ 1/env1 ]
                        occur[tmp][3] += [ env2 ]
                
                for tp in quasi_type:
                    if occur[tp][0] == 0:
                        quasi_feat += [0,0,0,0]
                    else:
                        mean_dis = mean(occur[tp][2])
                        dists = occur[tp][1]
                        vals = occur[tp][2]

                        count1 = dist1 = 0
                        count2 = dist2 = 0

                        for d, v in zip(dists, vals):
                            if v <= mean_dis:
                                count1 += 1
                                dist1 += d
                            else:
                                count2 += 1
                                dist2 += d

                        if count1 == 0:
                            if count2 == 0:
                                quasi_feat += [0, 0, 0, 0]
                            else:
                                quasi_feat += [0, 0, count2, dist2 / count2]
                        else:
                            if count2 == 0:
                                quasi_feat += [count1, dist1 / count1, 0, 0]
                            else:
                                quasi_feat += [count1, dist1 / count1, count2, dist2 / count2]


            return quasi_feat  
    #MIX
    def get_quasi_fragmental_desp_ext6(self, bins = None):
            """
            Compute extended quasi fragmental descriptors.
            Parameters:
                bins - distance bins for extracting qf descriptors
            """
            if bins is not None:
                self.contact_bins = bins
            else:
                self.contact_bins = [(0, 12)]

            # list atom types ----------------------------------------------------
            pro_pool = [6, 7, 8, 16]
            lig_pool = [6, 7, 8, 9, 15, 16, 17, 35, 53]
            # get list of atom-pair types ----------------------------------------
            quasi_type = list(itertools.product(pro_pool, lig_pool))        
            quasi_feat = []
            debug = defaultdict(int)
            for cbin in self.contact_bins:
                occur = {}
                for tp in quasi_type:
                    occur[tp] = [0, [], [], []]
                # check the contacts one by one ----------------------------------
                contacts = np.nonzero((self.pd >= cbin[0]) & (self.pd < cbin[1]))
                conts = [(int(i), int(j)) for (i, j) in zip(contacts[0], contacts[1])]
                distances = [self.pd[i, j] for (i, j) in conts]
                for ind in range(len(conts)):
                    cont = conts[ind]
                    cur_dist = distances[ind]
                    atm1 = self.pro[1].GetAtomWithIdx(cont[0])
                    atm2 = self.lig[1].GetAtomWithIdx(cont[1])                
                    atm1_an = atm1.GetAtomicNum()
                    atm2_an = atm2.GetAtomicNum()
                    
                    tmp = (atm1_an, atm2_an)
                    if tmp in quasi_type:
                        occur[tmp][0] += 1
                        env1 = self.protein_env.get(cont[0], 1)
                        env2 = self.ligand_env.get(cont[1], 1)
                        debug[(env1,env2)] += 1
                        occur[tmp][1] += [cur_dist]
                        occur[tmp][2] += [ 1/env1 ]
                        occur[tmp][3] += [ env2 ]
                
                for tp in quasi_type:
                    if occur[tp][0] == 0:
                        quasi_feat += [0,0,0,0]
                    else:
                        quasi_feat += [occur[tp][0], mean(occur[tp][1]),mean(occur[tp][2]),mean(occur[tp][3])]

            return quasi_feat  
                


            return quasi_feat  
    

In [31]:

def _featurize_complex(pdbinfo, log_message, para,model):
    """Featurizes a complex.
    First initializes an SIFt object, and then calculates the intlst.
    """   
    logging.info(log_message)
    protein_path = pdbinfo['fn_pro_PDB']
    pkl_path = protein_path.replace('_protein.pdb', '_sift.pkl')

# Check if the corresponding pickle file exists
    if os.path.isfile(pkl_path):
        pass
    else:
        return None

    # try:
    try :
        with open(pkl_path, 'rb') as f:
            pickled_sift_obj = pickle.load(f)
    except:
        return None
    sift = SIFt(pickled_sift_obj)
    if sift is None:
        print("error in retriving object")
        return None
    desp = None
    if model == 1:
        desp= sift.get_quasi_fragmental_desp_ext1(bins = para['bins'])
    if model == 2:
        desp= sift.get_quasi_fragmental_desp_ext2(bins = para['bins'])
    if model == 3:
        desp= sift.get_quasi_fragmental_desp_ext3(bins = para['bins'])
    if model == 4:
        desp= sift.get_quasi_fragmental_desp_ext4(bins = para['bins'])
    if model == 5:
        desp= sift.get_quasi_fragmental_desp_ext5(bins = para['bins'])
    if model == 6:
        desp= sift.get_quasi_fragmental_desp_ext6(bins = para['bins'])

    if (desp is None):
        print("wtf")
    return desp
#     except:
#         print("Error during featurization:")
# #        traceback.print_exc()
#         return None


In [32]:
def featurize_complexes(ligand_pdbfiles, 
                        protein_pdbfiles,
                        ligand_mol2files, 
                        protein_mol2files,
                        pdbids,model,
                        para = {'addH': True, 'sant': True,
                                'cutoff': 4.5, 'includeH': False, 'ifp_type': 'sift', 'count': 1,
                                'intlst': ['contact', 'backbone', 'sidechain', 'polar', 'nonpolar', 'donor', 'acceptor'],
                                'inttype': 'CH'}
#                        para = {'addH': True, 'sant': True,
#                                'cutoff': 4.5, 'ifp_type': 'rfscore',
#                                'bins': None, # use default contact bins
#                                'solo': 0, 'lst': []}
#                        para = {'addH': True, 'sant': False,
#                                'cutoff': 4.5, 'ifp_type': 'splif',
#                                'ecfp_radius': [1, 1],
#                                'base_prop': ['AtomicNumber', 'TotalConnections', 'HCount', 'HeavyNeighborCount', 'FormalCharge'],
#                                'folding_para': {'power': np.arange(6, 8, 1), 'counts': 1}}
                        ):
    """Obtains SIFts of a group of complexes.
    Parameters:
        ligand_pdbfiles - a list of ligand pdb files
        protein_pdbfiles - a list of protein pdb files
        ligand_mol2files - a list of ligand mol2 files
        protein_mol2files - a list of protein mol2 files
        pdbids - a list of pdb ids for the complexes under processing
        para - parameters for constructing SIFts
    Returns a list of the SIFts and the indices of failed complexes.
    """
    pool = multiprocessing.Pool(processes =8)
    results = []
    feat = []
    failures = []
    info = zip(ligand_pdbfiles, protein_pdbfiles, ligand_mol2files, protein_mol2files, pdbids)
    for i, (lig_pdbfile, pro_pdbfile, lig_mol2file, pro_mol2file, pdbid) in enumerate(info):
        #print(i)
        log_message = "Featurizing %d / %d complex..." % (i, len(pdbids))
        pdbinfo = {'fn_pro_PDB': pro_pdbfile, 'fn_lig_PDB': lig_pdbfile, 
                   'fn_pro_MOL2': pro_mol2file, 'fn_lig_MOL2': lig_mol2file, 'pdbid': pdbid}
        results.append(pool.apply_async(_featurize_complex, (pdbinfo, log_message, para,model)))      
    pool.close()  
    pool.join()
    for ind, result in enumerate(results):
        new_sift = result.get()
        if new_sift is None:
            failures.append(ind)
        else:
            feat.append(new_sift)
    
    return feat, failures



In [33]:
def loaddata_fromPDBbind_v2020(data_dir,modele,
                               subset = "refined",
                               select_target = ['HIV-1 PROTEASE'],
                               randsplit_ratio = [0.5, 0.25, 0.25],
                               para = {'addH': True, 'sant': True,
                                       'cutoff': 4.5, 'ifp_type': 'rfscore',
                                       'bins': None, 'lst': [], # use default contact bins
                                       'solo': 0},
                               rand_seed = 123):
    dt_dir = data_dir
    os.chdir(dt_dir)
    
    # -----------------------------------------------------------------------------------------------------------------------
    # get folder containing the structural data and the index dataframe containing the pdbs/affinities
    # -----------------------------------------------------------------------------------------------------------------------
    data_folder_dict = {'refined': 'data/'}
    data_index_dict = {'refined': 'indexes/rs_index.csv'}
    for s in ['casf2016']:
        data_folder_dict[s] = 'data/'
        data_index_dict[s] = 'indexes/' + s + '_index.csv'

    if subset in ['casf2016']:
        cur_fd = data_folder_dict[subset]
        df_index = pd.read_csv(data_index_dict[subset])
    else:
        cur_fd = data_folder_dict['refined']
        df_index_all = pd.read_csv(data_index_dict['refined'])
        df_index = df_index_all
        
    # further filter the dataframe according to select_target
    pdbs_selected = df_index['id'].tolist()
    labels_selected = df_index['affinity'].astype(float).tolist()
            
    # obtain the pdb and mol2 filename lists for the complexes -----------------------------------------------------------------------
    print('Generate file names................................................................')
    protein_pdbfiles = []
    ligand_pdbfiles = []
    protein_mol2files = []
    ligand_mol2files = []
    for pdb in pdbs_selected:
        protein_pdbfiles += [os.path.join(dt_dir, cur_fd, pdb, "%s_protein.pdb" % pdb)]
        protein_mol2files += [os.path.join(dt_dir, cur_fd, pdb, "%s_protein.mol2" % pdb)]
        ligand_pdbfiles += [os.path.join(dt_dir, cur_fd, pdb, "%s_ligand.pdb" % pdb)]
        ligand_mol2files += [os.path.join(dt_dir, cur_fd, pdb, "%s_ligand.mol2" % pdb)]  
        
    # featurize complexes using SIFts and split them into train, validation and test sets --------------------------------------------
    print('Begin to featurize dataset....................................................................')
    feat_t1 = time.time()
    feat, flrs = featurize_complexes(ligand_pdbfiles = ligand_pdbfiles, 
                                     protein_pdbfiles = protein_pdbfiles,
                                     ligand_mol2files = ligand_mol2files, 
                                     protein_mol2files = protein_mol2files,
                                     pdbids = pdbs_selected,model = modele,
                                     para = para)
    # Delete labels and ids for failing elements
    labels_lft = np.delete(labels_selected, flrs)
    return [np.array(feat),np.array(labels_lft)]


In [34]:
def cross_validate_models(X, y, n_splits=10, random_state=42):
    """
    Perform K-Fold CV for three regression models and return CV metrics.
    """
    models = {
        'RandomForest9': RandomForestRegressor(random_state=random_state,n_estimators = 650),
        'GradientBoosting9': GradientBoostingRegressor(random_state=random_state,n_estimators=700),
        "svr_poly1" : SVR(kernel='poly', degree=6, C=1.0, epsilon=0.1, coef0=1),
        "XGB2" : XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth=10),
        "Linear" : LinearRegression()
        
    }
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    cv_results = {name: {'pearson': [], 'rmse': []} for name in models}
    # return cv_results, models
    for name, model in models.items():
        for train_idx, val_idx in kf.split(X):

            X_tr, X_val = X[train_idx], X[val_idx]
            y_tr, y_val = y[train_idx], y[val_idx]
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_val)
            # Metrics
            r, _ = pearsonr(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            cv_results[name]['pearson'].append(r)
            cv_results[name]['rmse'].append(rmse)

        # Aggregate
        pearson_vals = cv_results[name]['pearson']
        rmse_vals    = cv_results[name]['rmse']

        pearson_mean = np.mean(pearson_vals)
        pearson_std  = np.std(pearson_vals)   # sample std
        rmse_mean    = np.mean(rmse_vals)
        rmse_std     = np.std(rmse_vals)      # sample std

        print(
            f"{name} CV -> "
            f"Pearson: {pearson_mean:.3f} ± {pearson_std:.3f}, "
            f"RMSE: {rmse_mean:.3f} ± {rmse_std:.3f}"
        )


    return cv_results, models


In [35]:

def train_and_test(X_train, y_train, X_test, y_test):
    # Load training data

    scaler = StandardScaler()
    X_scaled_train = scaler.fit_transform(X_train)

    # 2. Fit PCA on the scaled training data and transform
    pca = PCA(n_components=0.95)  # retain 95% of variance
    X_pca_train = pca.fit_transform(X_scaled_train)

    # 3. Transform test data with the *same* scaler and PCA
    X_scaled_test = scaler.transform(X_test)
    X_pca_test   = pca.transform(X_scaled_test)
    
    print(f"Training samples: {X_pca_train.shape[0]}, features: {X_pca_train.shape[1]}")

    # Cross-validate
    cv_results, models = cross_validate_models(X_pca_train, y_train)

    # Train final models on all training data and evaluate on test set
    print(f"Test samples: {X_pca_test.shape[0]}")

    for name, model in models.items():
        model.fit(X_pca_train, y_train)
        y_pred = model.predict(X_pca_train)


        
        # f = open('/home/ansh-meshram/Desktop/work/bio_project/pred_data.txt', 'a')
        # # Write the variables to the file
        # f.write(f"Name: {name}\n")
        # f.write(f"pred 1: {y_pred}\n")
        # f.write(f"test 2: {y_test}\n")
        # # Close the file
        # f.close()


        r, _ = pearsonr(y_train, y_pred)
        rmse = np.sqrt(mean_squared_error(y_train, y_pred))
        print(f" TRAIN :: {name} Test -> Pearson: {r:.3f}, RMSE: {rmse:.3f}")


    for name, model in models.items():
        model.fit(X_pca_train, y_train)
        y_pred = model.predict(X_pca_test)


        # f = open('/home/ansh-meshram/Desktop/work/bio_project/pred_data.txt', 'a')
        # # Write the variables to the file
        # f.write(f"Name: {name}\n")
        # f.write(f"pred 1: {y_pred}\n")
        # f.write(f"test 2: {y_test}\n")
        # # Close the file
        # f.close()
        r, _ = pearsonr(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f"TEST :: {name} Test -> Pearson: {r:.3f}, RMSE: {rmse:.3f}")



In [36]:


def round_robin_split_and_plot_correlation(X, y, N):
    """
    Splits the feature matrix (X) into N groups in round-robin fashion,
    computes the correlation of each feature in the group with the target (y),
    and plots a bar chart of correlations for each group.
    
    Parameters:
    - X: 2D numpy array (num_samples, num_features) containing feature values
    - y: List or numpy array of target values (num_samples,)
    - N: Number of groups to split the features into
    """
    # Step 1: Divide the feature matrix (X) into N groups using round-robin
    num_features = X.shape[1]
    feature_groups = [[] for _ in range(N)]  # Create empty lists for each group
    
    # Round-robin assignment of features to groups
    for i in range(num_features):
        feature_groups[i % N].append(X[:, i])  # Assign each feature to a group
    
    # Convert each group of features into a matrix (stack columns together)
    for i in range(N):
        feature_groups[i] = np.column_stack(feature_groups[i])
    
    # Step 2: Compute correlation and plot for each group
    for i, group in enumerate(feature_groups):
        # Compute the Pearson correlation between each feature in the group and the target (y)
        correlations = [np.corrcoef(group[:, j], y)[0, 1] for j in range(group.shape[1])]
        
        # Create a bar plot of the correlations
        plt.figure(figsize=(12, 6))
        plt.bar(range(group.shape[1]), correlations, color='lightblue')
        plt.title(f'Group {i + 1} - Correlation of Features with Target')
        plt.xlabel('Feature Index')
        plt.ylabel('Pearson Correlation')
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:
sd = 50
np.random.seed(sd)
data_dir = "/home/ansh-meshram/Desktop/work/bio_project/PDBbind"
spr_train = [0.9, 0.1, 0]
spr_test = [1, 0, 0]
trparadict = {'rf': {'rf_n_estimators': np.arange(300, 800, 100).tolist()},
              'gb': {'gb_n_estimators': np.arange(300, 800, 100).tolist()},
               'sv' : {}  
            }
datasets = ['rs-casf2016-csarhiq', 'casf2016', 'csarhiqS1', 'csarhiqS2', 'csarhiqS3']
# 1.1. featurize datasets -----------------------------------------------------------------------------------
#################################### IMCPs #######################################################################
ifptp = 'rfscore_ext'
cur_cutoff = 12
bins = [[(0,12)],[(0,6),(6,12)],[(0,4),(4,8),(8,12)]]
                      

for cur_bins in bins:
        para = {'addH': True, 'sant': False,
        'cutoff': cur_cutoff, 'ifp_type': ifptp,
        'bins': cur_bins}   
        for modelno in range(3,7):
                print("Model Number =>",modelno)                
                feat_dt_train = loaddata_fromPDBbind_v2020(data_dir = data_dir,modele = modelno,
                                                        subset = datasets[0],
                                                        select_target = None, 
                                                        randsplit_ratio = spr_train,
                                                        para = para,
                                                        rand_seed = sd)
                feat_dt_test1 = loaddata_fromPDBbind_v2020(data_dir = data_dir,modele = modelno, 
                                                        subset = datasets[1], 
                                                        select_target = None, 
                                                        randsplit_ratio = spr_test,
                                                        para = para,
                                                        rand_seed = sd)

                # 1.2 Model parameterization and validation -----------------------------------------------------------------
                # results saved in 'res.txt' in the data folder ------------------------------------------------------------
                group_val = 0
                if modelno == 1:
                        group_val = 2
                elif modelno == 2:
                        group_val = 1
                elif modelno == 3:
                        group_val = 2
                elif modelno == 4:
                        group_val = 3
                elif modelno == 5:
                        group_val = 4
                else:
                        group_val = 4
                #round_robin_split_and_plot_correlation(feat_dt_train[0],feat_dt_train[1],group_val)
                train_and_test(feat_dt_train[0],feat_dt_train[1],feat_dt_test1[0],feat_dt_test1[1],)



Model Number => 3
Generate file names................................................................
Begin to featurize dataset....................................................................
Generate file names................................................................
Begin to featurize dataset....................................................................
Training samples: 4803, features: 23
RandomForest9 CV -> Pearson: 0.678 ± 0.027, RMSE: 1.432 ± 0.050
GradientBoosting9 CV -> Pearson: 0.639 ± 0.031, RMSE: 1.500 ± 0.042
svr_poly1 CV -> Pearson: 0.249 ± 0.184, RMSE: 26.044 ± 44.921
XGB2 CV -> Pearson: 0.658 ± 0.025, RMSE: 1.470 ± 0.047
Linear CV -> Pearson: 0.573 ± 0.029, RMSE: 1.598 ± 0.044
Test samples: 497
 TRAIN :: RandomForest9 Test -> Pearson: 0.976, RMSE: 0.528
 TRAIN :: GradientBoosting9 Test -> Pearson: 0.889, RMSE: 0.936
 TRAIN :: svr_poly1 Test -> Pearson: 0.771, RMSE: 1.242
 TRAIN :: XGB2 Test -> Pearson: 0.997, RMSE: 0.167
 TRAIN :: Linear Test -> Pearson